<a href="https://colab.research.google.com/github/wizard339/article_finder/blob/main/report_metr_exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [109]:
import pandas as pd
import numpy as np
import re
import zipfile
from collections import namedtuple

In [111]:
data = pd.read_excel('/content/Для анализа.xlsx', index_col='№ п/п')
data.head()

,Дата получения документа,Дата возврата документа,Номер документа,Название изделия/обозначение,"Отдел, разработчик",Количество форматов А4,Фамилия эксперта,"Проект, тема, литера/ примечания",Замечания (шифры),Повторные замечания (шифры)
№ п/п,,,,,,,,,,
1,2024-01-09,2024-01-09,ИТКС.733421.005,Основание,"КО, Авилов",8,Марусина,NaN,NaN,NaN
2,2024-01-09,2024-01-09,ИТКС.402132.032 ГЧ,Блок чувствительных элементов БЧЭ-23,"КО, Авилов",4,Марусина,NaN,NaN,NaN
3,2024-01-09,2024-01-09,ИТКС.402132.032 СБ,Блок чувствительных элементов БЧЭ-23,"КО, Авилов",8,Марусина,NaN,NaN,NaN
4,2024-01-10,2024-01-11,ИТКС.738-23,КМИВ.402113.007 ГВК-6-1,"ОРСДВ, Селицкая",1,Иванов,NaN,NaN,NaN
5,2024-01-10,2024-01-11,КМИВ.713352.004 Д2,КМИВ.402113.007 ГВК-6-1,"ОРСДВ, Селицкая",9,Иванов,NaN,"5, 21",21.0


# 1 Обработка пропусков в данных

In [112]:
# заполняем пропуски в столбце 'Отдел, разработчик' значениями предыдущей строки
data['Отдел, разработчик'] = data['Отдел, разработчик'].fillna(method='ffill')

In [113]:
data['Дата получения документа'] = data['Дата получения документа'].fillna(method='ffill')
data['Дата возврата документа'] = data['Дата возврата документа'].fillna(data['Дата получения документа'])

In [114]:
data[' Замечания (шифры)'] = data[' Замечания (шифры)'].fillna(0)
data['Повторные замечания (шифры)'] = data['Повторные замечания (шифры)'].fillna(0)

# 2 Создание дополнительных столбцов

In [115]:
data['Срок проверки'] = data['Дата возврата документа'] - data['Дата получения документа'] + pd.offsets.Day(1)

In [116]:
data['Разработчик'] = ''

In [117]:
# разделяем значения из столбца 'Отдел, разработчик' на 2 значения
for i in range(data['Отдел, разработчик'].shape[0]):
  data['Отдел, разработчик'].iloc[i] = data['Отдел, разработчик'].iloc[i].split(',')
  for s in data['Отдел, разработчик'].iloc[i]:
    s = s.strip()
    data['Разработчик'].iloc[i] = data['Отдел, разработчик'].iloc[i][1].strip()
  data['Отдел, разработчик'].iloc[i] = data['Отдел, разработчик'].iloc[i][0]

Выходные данные были обрезаны до нескольких последних строк (5000).
<ipython-input-117-65b288692923>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Отдел, разработчик'].iloc[i] = data['Отдел, разработчик'].iloc[i].split(',')
<ipython-input-117-65b288692923>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Разработчик'].iloc[i] = data['Отдел, разработчик'].iloc[i][1].strip()
<ipython-input-117-65b288692923>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

In [118]:
# исправляем типичные ошибки в названиях отделов + актуализируем названия
data['Отдел, разработчик'] = data['Отдел, разработчик'].replace(['ОРМД', 'ОРМС'],'ОРМ')
data['Отдел, разработчик'] = data['Отдел, разработчик'].replace('ОРЛГ', 'ОРИС')
data['Отдел, разработчик'] = data['Отдел, разработчик'].replace('ОРСТВГ', 'ОРС ТВГ')
data['Отдел, разработчик'] = data['Отдел, разработчик'].replace(['ОРСДВ', 'ОРС ДВ'], 'ОРС ДНГ и ВОГ')

data['Разработчик'] = data['Разработчик'].replace(['Докторова', 'Важнова'], 'Важнова (Докторова)')
data['Фамилия эксперта'] = data['Фамилия эксперта'].replace(['Докторова', 'Важнова'], 'Важнова (Докторова)')
data['Фамилия эксперта'] = data['Фамилия эксперта'].replace(['Мальцева', 'Мацюк'], 'Мацюк (Мальцева)')

In [119]:
# добавляем к фамилии разработчика суффикс с названием его отдела
for i in range(data['Разработчик'].shape[0]):
  data['Разработчик'].iloc[i] = data['Разработчик'].iloc[i] + '_' + data['Отдел, разработчик'].iloc[i]

<ipython-input-119-7ef14eeaf7c5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Разработчик'].iloc[i] = data['Разработчик'].iloc[i] + '_' + data['Отдел, разработчик'].iloc[i]
<ipython-input-119-7ef14eeaf7c5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Разработчик'].iloc[i] = data['Разработчик'].iloc[i] + '_' + data['Отдел, разработчик'].iloc[i]
<ipython-input-119-7ef14eeaf7c5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

# 3 Обработка столбца с замечаниями

In [120]:
# для удобства дальнейшей обработки создаем список шифров
for i in range(data[' Замечания (шифры)'].shape[0]):
   a = [s.strip() for s in re.split('[,.]', str(data[' Замечания (шифры)'].iloc[i]))]
   data[' Замечания (шифры)'].iloc[i] = a

for i in range(data['Повторные замечания (шифры)'].shape[0]):
   a = [s.strip() for s in re.split('[,.]', str(data['Повторные замечания (шифры)'].iloc[i]))]
   data['Повторные замечания (шифры)'].iloc[i] = a

<ipython-input-120-916e1521561e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[' Замечания (шифры)'].iloc[i] = a
<ipython-input-120-916e1521561e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[' Замечания (шифры)'].iloc[i] = a
<ipython-input-120-916e1521561e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[' Замечания (шифры)'].iloc[i] = a
<ipython-input-120-916e1521561e>:4: SettingWithCopyWarning: 
A value is tryi

In [121]:
data.sample(10)

,Дата получения документа,Дата возврата документа,Номер документа,Название изделия/обозначение,"Отдел, разработчик",Количество форматов А4,Фамилия эксперта,"Проект, тема, литера/ примечания",Замечания (шифры),Повторные замечания (шифры),Срок проверки,Разработчик
№ п/п,,,,,,,,,,,,
195,2024-01-29,2024-01-29,ИТКС.754312.075,Знак заводской,КО,2,Марусина,NaN,[0],"[0, 0]",1 days,Авилов_КО
429,2024-03-01,2024-03-01,ИТКС.123-24,ИТКС.687281.352 Плата,КО,1,Марусина,NaN,[0],"[0, 0]",1 days,Петров_КО
391,2024-02-21,2024-02-21,КМИВ.687417.018 Д,КМИВ.687417.018 Плата масштабных резисторов,ОРС ДНГ и ВОГ,1,Иванов,NaN,[0],"[0, 0]",1 days,Селицкая_ОРС ДНГ и ВОГ
374,2024-02-20,2024-02-21,ИТКС.105-24,ИТКС.402132.020 ЭЧ-13,КО,1,Иванов,NaN,[0],"[0, 0]",2 days,Крючкова_КО
579,2024-03-21,2024-03-21,ИТКС.191-24,ИТКС.433724.001 ГМ-3,ОРИС,1,Иванов,NaN,[0],"[0, 0]",1 days,Демкин_ОРИС
58,2024-01-16,2024-01-17,ИТКС.461534.019-01 Д,ИТКС.461534.019-01 ЦПС-18-1,ОРЭ,3,Иванов,NaN,[0],"[0, 0]",2 days,Казакова_ОРЭ
339,2024-02-12,2024-02-14,ИТКС.794774.011 ПС,ИТКС.794774.011 АСК резонатора,ОРС ТВГ,15,Иванов,NaN,[0],"[0, 0]",3 days,Высоцкая_ОРС ТВГ
13,2024-01-09,2024-01-12,NaN,"Перечень ДСЕ, ПКИ подлежащих входному контролю...",ОГТ,7,Иванов,NaN,[3],"[0, 0]",4 days,Черняев_ОГТ
26,2024-01-12,2024-01-12,ИТКС.711141.043,Кольцо,КО,1,Марусина,NaN,[0],"[0, 0]",1 days,Петров_КО


In [122]:
# функция для подсчета количества ошибок по отделам

def count_codes(data: pd.DataFrame, department: str) -> dict:
  # словарь для хранения количества замечаний по каждому шифру
  codes = dict(zip(np.arange(0, 31), np.zeros(30)))

  # цикл по строкам датафрейма  с помощью указателя i
  for i in range(data.shape[0]):
    # если строка относится к нужному отделу
    if data['Отдел, разработчик'].iloc[i] == department:
      # берем значение из столбца с замечаниями

      for c in data[' Замечания (шифры)'].iloc[i]:
        # добавляем к счетчику шифров
        codes[int(c)] += 1
      for c in data['Повторные замечания (шифры)'].iloc[i]:
        # добавляем к счетчику шифров за исключением 0 шифра
        if int(c) != 0:
          codes[int(c)] += 1

  return codes

In [123]:
# функция для подсчета количества ошибок по разработчику

def count_codes_by_person(data: pd.DataFrame, person: str) -> dict:
  # словарь для хранения количества замечаний по каждому шифру
  codes = dict(zip(np.arange(0, 31), np.zeros(30)))

  # цикл по строкам датафрейма  с помощью указателя i
  for i in range(data.shape[0]):
    # если строка относится к нужному разработчику
    if data['Разработчик'].iloc[i] == person:
      # берем значение из столбца с замечаниями

      for c in data[' Замечания (шифры)'].iloc[i]:
        # добавляем к счетчику шифров
        codes[int(c)] += 1
      for c in data['Повторные замечания (шифры)'].iloc[i]:
        # добавляем к счетчику шифров за исключением 0 шифра
        if int(c) != 0:
          codes[int(c)] += 1

  return codes

# 4 Формирование отчетов

In [124]:
def make_main_report(data: pd.DataFrame) -> pd.DataFrame:
  report = pd.DataFrame(columns=['Проверено документов, шт',
                                   'Проверено листов А4, шт',
                                   'Всего замечаний, шт',
                                   'Доля документов с замечаниями, %',
                                   'Количество замечаний на 100 листов А4, шт'])
  report['Проверено документов, шт'] = data.groupby(['Отдел, разработчик']).count()['Дата получения документа']
  report['Проверено листов А4, шт'] = data[['Отдел, разработчик', 'Количество форматов А4']].groupby(['Отдел, разработчик']).sum()

  # в цикле создаем вспомогательные переменные для подсчета замечаний и показателей
  for name in report.index.to_list():
    # создаем словарь для каждого отдела с количеством замечаний
    codes = count_codes(data, name)
    # переменная для хранения количества замечаний без учета шифра '17'
    total_without_17 = 0
    # переменная для хранения количества замечаний с шифром '17'
    count_17 = codes[17]
    # переменная для хранения количества документов без замечаний
    without_mistakes = codes[0]
    for c in codes.keys():
      if c not in (0, 17):
        total_without_17 += codes[c]
    # переменная для хранения общего числа замечаний
    total_count = total_without_17 + count_17

    report.loc[name, 'Всего замечаний, шт'] = total_count
    report.loc[name, 'Доля документов с замечаниями, %'] = np.round(((1 - (without_mistakes / report.loc[name, 'Проверено документов, шт'])) * 100), 1)
    report.loc[name, 'Количество замечаний на 100 листов А4, шт'] = np.round(((total_count / report.loc[name, 'Проверено листов А4, шт']) * 100), 1)

  # создаем переменные для хранения сводных показателей по предприятию
  all_docs = np.sum(report['Проверено документов, шт'])
  all_pages = np.sum(report['Проверено листов А4, шт'])
  all_mistakes = np.sum(report['Всего замечаний, шт'])
  all_docs_with_mistakes = np.sum(report['Проверено документов, шт'] * (report['Доля документов с замечаниями, %'] / 100))

  # добавляем в датафрейм строку со сводными показателями
  report.loc['Общая статистика по предприятию'] = [all_docs, all_pages, all_mistakes, np.round((all_docs_with_mistakes / all_docs)*100, 1), np.round((all_mistakes / all_pages)*100, 1)]

  return report

In [125]:
def make_codes_report(data: pd.DataFrame) -> pd.DataFrame:
  Codes = namedtuple('Codes', ['department', 'codes'])
  counts_of_codes = [Codes(dp, count_codes(data, dp)) for dp in data['Отдел, разработчик'].unique()]

  cols = list(data['Отдел, разработчик'].unique())
  cols.insert(0, 'Шифр')
  report = pd.DataFrame(columns=[cols])

  for nt in counts_of_codes:
    report[nt.department] = nt.codes.values()

  report['Шифр'] = report.index

  return report

In [126]:
def make_codes_by_person_report(data: pd.DataFrame) -> pd.DataFrame:
  Codes = namedtuple('Codes', ['person', 'codes'])
  counts_of_codes = [Codes(pers, count_codes_by_person(data, pers)) for pers in data['Разработчик'].unique()]

  cols = list(data['Разработчик'].unique())
  cols.insert(0, 'Шифр')
  report = pd.DataFrame(columns=[cols])

  for nt in counts_of_codes:
    report[nt.person] = nt.codes.values()

  report['Шифр'] = report.index
  report = report.T
  report = report.drop('Шифр', axis=0)

  report.insert(0, 'Отдел', [report.index.values[i][0].split('_')[1] for i in range(report.index.shape[0])])

  return report

In [127]:
def make_time_report(data: pd.DataFrame) -> pd.DataFrame:
  report = pd.DataFrame(data['Срок проверки'].value_counts())
  return report

# 5 Деление датасета на части

In [128]:
# разделим датасет на части по кварталам и полугодиям
data_1q = data[(data['Дата получения документа'] > '2024-01-01') & (data['Дата получения документа'] < '2024-04-01')]
data_2q = data[(data['Дата получения документа'] >= '2024-04-01') & (data['Дата получения документа'] < '2024-07-01')]
data_3q = data[(data['Дата получения документа'] >= '2024-07-01') & (data['Дата получения документа'] < '2024-10-01')]
data_4q = data[(data['Дата получения документа'] >= '2024-10-01') & (data['Дата получения документа'] < '2025-01-01')]
data_1_half_year = data[(data['Дата получения документа'] >= '2024-01-01') & (data['Дата получения документа'] < '2024-07-01')]
data_2_half_year = data[(data['Дата получения документа'] >= '2024-07-01') & (data['Дата получения документа'] < '2025-01-01')]
data_full = data[(data['Дата получения документа'] >= '2024-01-01') & (data['Дата получения документа'] < '2025-01-01')]
# data_list = [data_1q, data_2q, data_3q, data_4q, data_1_half_year, data_2_half_year, data_full]
data_list = [data_1q]

In [129]:
# создадим аттрибуты с именем датасета
data_1q.name = '1_квартал'
data_2q.name = '2_квартал'
data_3q.name = '3_квартал'
data_4q.name = '4_квартал'
data_1_half_year.name = '1_полугодие'
data_2_half_year.name = '2_полугодие'
data_full.name = 'Весь_год'

In [130]:
print(f'1 квартал: {data_1q.shape} \n2 квартал: {data_2q.shape} \n3 квартал: {data_3q.shape}  \n4 квартал: {data_4q.shape} \n1 полугодие: {data_1_half_year.shape}  \n2 полугодие: {data_2_half_year.shape} \nВесь год: {data_full.shape}')

1 квартал: (660, 12) 
2 квартал: (0, 12) 
3 квартал: (0, 12)  
4 квартал: (0, 12) 
1 полугодие: (660, 12)  
2 полугодие: (0, 12) 
Весь год: (660, 12)


# 6 Сохранение отчетов

In [131]:
for d in data_list:
  with zipfile.ZipFile(f'{d.name}.zip', 'w') as zf:
    with zf.open(f'{d.name}_основной_отчет.xlsx', 'w') as buffer:
      with pd.ExcelWriter(buffer) as writer:
        df = make_main_report(d)
        df.to_excel(writer)

    with zf.open(f'{d.name}_основной_по_ошибкам.xlsx', 'w') as buffer:
      with pd.ExcelWriter(buffer) as writer:
        df = make_codes_report(d)
        df.to_excel(writer)

    with zf.open(f'{d.name}_ошибки_по_разработчику.xlsx', 'w') as buffer:
      with pd.ExcelWriter(buffer) as writer:
        df = make_codes_by_person_report(d)
        df.to_excel(writer)

    with zf.open(f'{d.name}_основной_по_срокам.xlsx', 'w') as buffer:
      with pd.ExcelWriter(buffer) as writer:
        df = make_time_report(d)
        df.to_excel(writer)


<ipython-input-126-b7c71240e927>:14: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  report = report.drop('Шифр', axis=0)
